This should create .csv files that can be imported into pandas for all the plotting routines. They are all loading/calculating the same thing in their respective notebooks, so this is an attempt to consilidate the repeated code to make one file that all plottings scripts can import. 

Import the necessary packages

In [1]:
import astropy as ap
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.modeling.models import custom_model
from astropy.cosmology import FlatLambdaCDM

In [2]:
import numpy as np
import math as m

In [3]:
import scipy as sp
from scipy import integrate

In [4]:
import uncertainties
from uncertainties import unumpy as unp

In [5]:
import pandas as pd

In [6]:
import os

In [7]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib as mp

In [8]:
cosmo=FlatLambdaCDM(H0=70.0,Om0=0.3)
fit=LevMarLSQFitter()
GAL_ALPHA=-1.1
LCBG_ALPHA=-1.0

In [9]:
@uncertainties.wrap
def integratethings(ps,ms,al=None,m=None):
    if al==None:
        al=-1.1
    if m==None:
        m=-18.5
    def integrand(X):
        return (0.4*np.log(10)*ps)*(10**(0.4*(al+1)*(ms-X)))*(np.e**(-np.power(10,0.4*(ms-X))))
    integral,abserr=sp.integrate.quad(integrand,-100,m)
    return integral

In [10]:
@custom_model
def schechter_func_gal(x,phistar=0.0056,mstar=-21):
    return (0.4*np.log(10)*phistar)*(10**(0.4*(GAL_ALPHA+1)*(mstar-x)))*(np.e**(-np.power(10,0.4*(mstar-x))))

In [11]:
@custom_model
def schechter_func_lcbg(x,phistar=0.0056,mstar=-21):
    return (0.4*np.log(10)*phistar)*(10**(0.4*(LCBG_ALPHA+1)*(mstar-x)))*(np.e**(-np.power(10,0.4*(mstar-x))))

In [12]:
def schechter_func_scipy_gal(x,phistar,mstar):
    return (0.4*np.log(10)*phistar)*(10**(0.4*(GAL_ALPHA+1)*(mstar-x)))*(np.e**(-np.power(10,0.4*(mstar-x))))

In [13]:
def schechter_func_scipy_lcbg(x,phistar,mstar):
    return (0.4*np.log(10)*phistar)*(10**(0.4*(LCBG_ALPHA+1)*(mstar-x)))*(np.e**(-np.power(10,0.4*(mstar-x))))

In [14]:
def ap_fitting_gal(DATAFRAME,MASK=None):
    if MASK is None:
        MASK=[]
    #******CREATE MASK ARRAY******
    NUMBDENS=np.ma.array(np.array(DATAFRAME['Number_Density']),mask=False)
    MBINAVE=np.ma.array(np.array(DATAFRAME['Average_Absolute_Magnitude']),mask=False)
    NUMBDENSERR=np.ma.array(np.array(DATAFRAME['Number_Density_Error']),mask=False)
    #******MASK ZEROS******
    NUMBDENS.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
    MBINAVE.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
    NUMBDENSERR.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
     #******MASK POINTS I DON'T LIKE******
    NUMBDENS.mask[MASK]=True
    MBINAVE.mask[MASK]=True
    NUMBDENSERR.mask[MASK]=True
    print(NUMBDENS.compressed())
    #******MODELLING******
    LFFIT_INIT=schechter_func_gal()
    LFFIT=fit(LFFIT_INIT,MBINAVE.compressed(),NUMBDENS.compressed(),weights=1/NUMBDENSERR.compressed())
    return LFFIT,NUMBDENS.mask

In [15]:
def ap_fitting_lcbg(DATAFRAME,MASK=None):
    if MASK is None:
        MASK=[]
    #******CREATE MASK ARRAY******
    NUMBDENS=np.ma.array(np.array(DATAFRAME['Number_Density']),mask=False)
    MBINAVE=np.ma.array(np.array(DATAFRAME['Average_Absolute_Magnitude']),mask=False)
    NUMBDENSERR=np.ma.array(np.array(DATAFRAME['Number_Density_Error']),mask=False)
    #******MASK ZEROS******
    NUMBDENS.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
    MBINAVE.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
    NUMBDENSERR.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
     #******MASK POINTS I DON'T LIKE******
    NUMBDENS.mask[MASK]=True
    MBINAVE.mask[MASK]=True
    NUMBDENSERR.mask[MASK]=True
    print(NUMBDENS.compressed())
    #******MODELLING******
    LFFIT_INIT=schechter_func_lcbg()
    LFFIT=fit(LFFIT_INIT,MBINAVE.compressed(),NUMBDENS.compressed(),weights=1/NUMBDENSERR.compressed())
    return LFFIT,NUMBDENS.mask

In [16]:
def sp_fitting_gal(DATAFRAME,MASK=None,INITARRAY=None):
    if MASK is None:
        MASK=[]
    if INITARRAY==None:
        INITARRAY=[0.0056,-21]
    #******CREATE MASK ARRAY******
    NUMBDENS=np.ma.array(np.array(DATAFRAME['Number_Density']),mask=False)
    MBINAVE=np.ma.array(np.array(DATAFRAME['Average_Absolute_Magnitude']),mask=False)
    NUMBDENSERR=np.ma.array(np.array(DATAFRAME['Number_Density_Error']),mask=False)
    #******MASK ZEROS******
    NUMBDENS.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
    MBINAVE.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
    NUMBDENSERR.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
     #******MASK POINTS I DON'T LIKE******
    NUMBDENS.mask[MASK]=True
    MBINAVE.mask[MASK]=True
    NUMBDENSERR.mask[MASK]=True
    #******MODELLING******
    LFFIT,LFCOV=sp.optimize.curve_fit(schechter_func_scipy_gal,MBINAVE.compressed(),NUMBDENS.compressed(),p0=INITARRAY,sigma=NUMBDENSERR.compressed())
    return LFFIT,LFCOV

In [17]:
def sp_fitting_lcbg(DATAFRAME,MASK=None,INITARRAY=None):
    if MASK is None:
        MASK=[]
    if INITARRAY==None:
        INITARRAY=[0.0056,-21]
    #******CREATE MASK ARRAY******
    NUMBDENS=np.ma.array(np.array(DATAFRAME['Number_Density']),mask=False)
    MBINAVE=np.ma.array(np.array(DATAFRAME['Average_Absolute_Magnitude']),mask=False)
    NUMBDENSERR=np.ma.array(np.array(DATAFRAME['Number_Density_Error']),mask=False)
    #******MASK ZEROS******
    NUMBDENS.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
    MBINAVE.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
    NUMBDENSERR.mask[DATAFRAME.loc[DATAFRAME.Number_Density==0].index]=True
     #******MASK POINTS I DON'T LIKE******
    NUMBDENS.mask[MASK]=True
    MBINAVE.mask[MASK]=True
    NUMBDENSERR.mask[MASK]=True
    #******MODELLING******
    LFFIT,LFCOV=sp.optimize.curve_fit(schechter_func_scipy_lcbg,MBINAVE.compressed(),NUMBDENS.compressed(),p0=INITARRAY,sigma=NUMBDENSERR.compressed())
    return LFFIT,LFCOV

In [18]:
CATALOG=pd.read_csv('/home/lrhunt/Astrodata/LCBG_Luminosity_Function/Original_Catalogs/Final_Catalogs/COSMOS_CONVERTED_CATALOG.csv')

In [19]:
Magnitude_Loop_Array=np.stack((np.arange(15,
                                         22.5+0.5,
                                         0.5)[0:len(np.arange(15,
                                                              22.5+0.5,
                                                              0.5))-1],
                               np.arange(15,
                                         22.5+0.5,
                                         0.5)[1:len(np.arange(15,
                                                              22.5+0.5,
                                                              0.5))]),
                              axis=-1)
print('********CALCULATING WEIGHTS********')
tf=open('weights.txt','w')
CATALOG['Spec_Weight']=np.nan
CATALOG['Color_Weight']=np.nan
CATALOG['Surface_Brightness_Weight']=np.nan
for magrange in Magnitude_Loop_Array:
    Num_Good_Spec=len(CATALOG.loc[(CATALOG.subaru_i_mag>magrange[0])&
                              (CATALOG.subaru_i_mag<magrange[1])&
                              (CATALOG.SG_MASTER==0)&
                              ((CATALOG.Z_USE==1)|
                               (CATALOG.Z_USE==2))])
    Num_Bad_Spec=len(CATALOG.loc[(CATALOG.subaru_i_mag>magrange[0])&
                             (CATALOG.subaru_i_mag<magrange[1])&
                             (CATALOG.SG_MASTER==0)&
                             ((CATALOG.Z_USE==3)|
                              (CATALOG.Z_USE==4))])
    Num_Good_Color=len(CATALOG.loc[(CATALOG.subaru_i_mag>magrange[0])&
                               (CATALOG.subaru_i_mag<magrange[1])&
                               (CATALOG.SG_MASTER==0)&
                               ((CATALOG.subaru_B_mag<100)&
                                (CATALOG.subaru_V_mag<100))])
    Num_Bad_Color=len(CATALOG.loc[(CATALOG.subaru_i_mag>magrange[0])&
                              (CATALOG.subaru_i_mag<magrange[1])&
                              (CATALOG.SG_MASTER==0)&
                              ((CATALOG.subaru_B_mag>100)|
                               (CATALOG.subaru_V_mag>100))])
    Num_Bad_Rh=len(CATALOG.loc[(CATALOG.subaru_i_mag>magrange[0])&
                           (CATALOG.subaru_i_mag<magrange[1])&
                           (CATALOG.SG_MASTER==0)&
                           (np.isnan(CATALOG.R_HALF_PIXELS))])
    Num_Good_Rh=len(CATALOG.loc[(CATALOG.subaru_i_mag>magrange[0])&
                            (CATALOG.subaru_i_mag<magrange[1])&
                            (CATALOG.SG_MASTER==0)&
                            (~np.isnan(CATALOG.R_HALF_PIXELS))])
    CATALOG.loc[(CATALOG.subaru_i_mag>magrange[0])&
                (CATALOG.subaru_i_mag<magrange[1])&
                (CATALOG.SG_MASTER==0),
                'Spec_Weight']=float(Num_Good_Spec+Num_Bad_Spec)/float(Num_Good_Spec)
    CATALOG.loc[(CATALOG.subaru_i_mag>magrange[0])&
                (CATALOG.subaru_i_mag<magrange[1])&
                (CATALOG.SG_MASTER==0),
                'Color_Weight']=float(Num_Good_Color+Num_Bad_Color)/float(Num_Good_Color)
    CATALOG.loc[(CATALOG.subaru_i_mag>magrange[0])&
                (CATALOG.subaru_i_mag<magrange[1])&
                (CATALOG.SG_MASTER==0),
                'Surface_Brightness_Weight']=float(Num_Good_Rh+Num_Bad_Rh)/float(Num_Good_Rh)
    print('Spec Weight = {} | Color Weight = {} | Surface Brightness Weight = {}'.format(
        np.round(float(Num_Good_Spec+Num_Bad_Spec)/float(Num_Good_Spec),4),
        np.round(float(Num_Good_Color+Num_Bad_Color)/float(Num_Good_Color),4),
        np.round(float(Num_Good_Rh+Num_Bad_Rh)/float(Num_Good_Rh),4)))

********CALCULATING WEIGHTS********
Spec Weight = 5.75 | Color Weight = 1.0 | Surface Brightness Weight = 2.3
Spec Weight = 5.5 | Color Weight = 1.0 | Surface Brightness Weight = 11.0
Spec Weight = 1.1818 | Color Weight = 1.0 | Surface Brightness Weight = 6.5
Spec Weight = 1.0435 | Color Weight = 1.0 | Surface Brightness Weight = 1.3333
Spec Weight = 1.0566 | Color Weight = 1.0 | Surface Brightness Weight = 2.0
Spec Weight = 1.2267 | Color Weight = 1.011 | Surface Brightness Weight = 1.84
Spec Weight = 1.218 | Color Weight = 1.0 | Surface Brightness Weight = 1.4595
Spec Weight = 1.1867 | Color Weight = 1.0 | Surface Brightness Weight = 1.3021
Spec Weight = 1.1932 | Color Weight = 1.0 | Surface Brightness Weight = 1.4015
Spec Weight = 1.2101 | Color Weight = 1.0 | Surface Brightness Weight = 1.33
Spec Weight = 1.2232 | Color Weight = 1.0 | Surface Brightness Weight = 1.2778
Spec Weight = 1.2264 | Color Weight = 1.0 | Surface Brightness Weight = 1.3045
Spec Weight = 1.2818 | Color Weight

In [20]:
LCBG_DIR='/home/lrhunt/Projects/LCBG_LUMINOSITY_FUNCTION/LCBGLFOUT/apmag225/zucca_z_bins/'
LCBG_LFS={}
LCBG_LFS_INDEX=[]
for file in os.listdir(LCBG_DIR):
    if file.endswith('csv'):
        LCBG_LFS[file.split('.csv')[0]]=pd.read_csv(LCBG_DIR+'/'+file)
        LCBG_LFS_INDEX.append(file.split('.csv')[0])

In [21]:
GAL_DIR='/home/lrhunt/Projects/LCBG_LUMINOSITY_FUNCTION/FULLLFOUT/apmag225/zucca_z_bins/'
GAL_LFS={}
GAL_LFS_INDEX=[]
for file in os.listdir(GAL_DIR):
    if file.endswith('csv'):
        GAL_LFS[file.split('.csv')[0]]=pd.read_csv(GAL_DIR+'/'+file)
        GAL_LFS_INDEX.append(file.split('.csv')[0])

In [22]:
GAL_BIAS_CUTOFF={GAL_LFS_INDEX[0]:-17,
                 GAL_LFS_INDEX[1]:-19,
                 GAL_LFS_INDEX[2]:-20,
                 GAL_LFS_INDEX[3]:-20.5,
                 GAL_LFS_INDEX[4]:-20,}
LCBG_BIAS_CUTOFF={LCBG_LFS_INDEX[0]:-17,
                 LCBG_LFS_INDEX[1]:-19,
                 LCBG_LFS_INDEX[2]:-20,
                 LCBG_LFS_INDEX[3]:-20.5,
                 LCBG_LFS_INDEX[4]:-20,}

In [23]:
LCBG_MASKS={LCBG_LFS_INDEX[0]:[0,1,2,14,15,16,17],LCBG_LFS_INDEX[1]:[0,1,2,10,11,12,13,14],LCBG_LFS_INDEX[2]:[0,1,2,8,9,10,11,12,13,14],LCBG_LFS_INDEX[3]:[0,1,7,8,9,10,11,12,13,14,15,16,17],LCBG_LFS_INDEX[4]:[0,1,12,13,14,15,16,17]}
GAL_MASKS={GAL_LFS_INDEX[0]:[0,1,2,14,15,16,17],GAL_LFS_INDEX[1]:[10,11,12,13,14],GAL_LFS_INDEX[2]:[0,1,8,9,10,11,12,13,14],GAL_LFS_INDEX[3]:[0,7,8,9,10,11,12,13,14,15,16,17],GAL_LFS_INDEX[4]:[0,1,12,13,14,15,16,17]}

In [24]:
LCBG_FITS_AP={}
LCBG_FITS_AP_INDEX=[]
for index in LCBG_LFS_INDEX:
    LCBG_FITS_AP[index.split('LF_')[-1]+'_ap_fit']=ap_fitting_lcbg(LCBG_LFS[index],MASK=LCBG_MASKS[index])
    LCBG_FITS_AP_INDEX.append(index.split('LF_')[-1]+'_ap_fit')
LCBG_FITS_SP={}
LCBG_FITS_SP_INDEX=[]
for index in LCBG_LFS_INDEX:
    LCBG_FITS_SP[index.split('LF_')[-1]+'_sp_fit']=sp_fitting_lcbg(LCBG_LFS[index],MASK=LCBG_MASKS[index])
    LCBG_FITS_SP_INDEX.append(index.split('LF_')[-1]+'_sp_fit')

[3.88309294e-05 2.09689260e-04 4.11921946e-04 6.44976535e-04
 7.19701437e-04 9.37092577e-04 1.03893955e-03]
[2.08197684e-05 9.91660177e-05 3.05321945e-04 5.64217530e-04
 8.95283052e-04 1.25464917e-03 1.38373013e-03]
[2.76136761e-05 1.90660449e-04 5.34830678e-04 9.93091424e-04
 1.65637663e-03]
[1.04639596e-05 6.19733206e-05 2.96790527e-04 8.27359142e-04
 1.45524778e-03]
[7.59453123e-06 2.46949014e-05 1.52430076e-04 4.34269252e-04
 8.17079068e-04 1.30345264e-03 1.25943924e-03 1.35632472e-03
 1.47079447e-03]


In [25]:
i=0
GAL_FITS_AP={}
GAL_FITS_AP_INDEX=[]
z_bin_dict={}
for index in GAL_LFS_INDEX:
    GAL_FITS_AP[index.split('LF_')[-1]+'_ap_fit']=ap_fitting_gal(GAL_LFS[index],MASK=GAL_MASKS[index])
    GAL_FITS_AP_INDEX.append(index.split('LF_')[-1]+'_ap_fit')
GAL_FITS_SP={}
GAL_FITS_SP_INDEX=[]
for index in GAL_LFS_INDEX:
    GAL_FITS_SP[index.split('LF_')[-1]+'_sp_fit']=sp_fitting_gal(GAL_LFS[index],MASK=GAL_MASKS[index])
    GAL_FITS_SP_INDEX.append(index.split('LF_')[-1]+'_sp_fit')
    z_bin_dict[i]='{}$<z\le${}'.format(index.split('_')[1],index.split('_')[2])
    i=i+1

[4.35204484e-05 3.90956996e-04 1.06433704e-03 1.89980956e-03
 3.17913630e-03 4.23473684e-03 4.78368528e-03 6.16771388e-03
 6.80547699e-03 6.55525726e-03 7.75368194e-03]
[3.90196265e-06 3.07966351e-05 9.60147277e-05 3.99841976e-04
 1.04194817e-03 2.01964815e-03 2.87975770e-03 3.56893989e-03
 4.47730044e-03]
[3.49547729e-05 2.32957366e-04 6.60515844e-04 1.66001337e-03
 2.53645098e-03 3.70743764e-03]
[5.37942022e-06 5.46053990e-05 3.24140254e-04 9.58093906e-04
 1.97237802e-03 2.83414351e-03]
[3.25823790e-05 1.62859190e-04 5.41621554e-04 1.36090978e-03
 2.27913103e-03 3.33186153e-03 3.56125077e-03 4.54626722e-03
 6.11419520e-03 9.00644107e-03]


In [26]:
LCBG_VAL_ERROR_TO_DATAFRAME={}
for index in LCBG_FITS_SP_INDEX:
    LCBG_VAL_ERROR_TO_DATAFRAME[
        index.split('_sp')[0]
    ]=[uncertainties.correlated_values(LCBG_FITS_SP[index][0],
                                       LCBG_FITS_SP[index][1])[0].nominal_value,
      uncertainties.correlated_values(LCBG_FITS_SP[index][0],
                                       LCBG_FITS_SP[index][1])[0].std_dev,
      uncertainties.correlated_values(LCBG_FITS_SP[index][0],
                                       LCBG_FITS_SP[index][1])[1].nominal_value,
      uncertainties.correlated_values(LCBG_FITS_SP[index][0],
                                       LCBG_FITS_SP[index][1])[1].std_dev]    

In [27]:
GAL_VAL_ERROR_TO_DATAFRAME={}
for index in GAL_FITS_SP_INDEX:
    GAL_VAL_ERROR_TO_DATAFRAME[
        index.split('_sp')[0]
    ]=[uncertainties.correlated_values(GAL_FITS_SP[index][0],
                                       GAL_FITS_SP[index][1])[0].nominal_value,
      uncertainties.correlated_values(GAL_FITS_SP[index][0],
                                       GAL_FITS_SP[index][1])[0].std_dev,
      uncertainties.correlated_values(GAL_FITS_SP[index][0],
                                       GAL_FITS_SP[index][1])[1].nominal_value,
      uncertainties.correlated_values(GAL_FITS_SP[index][0],
                                       GAL_FITS_SP[index][1])[1].std_dev]    

In [28]:
GAL_FIT_VALS=pd.DataFrame.from_dict(GAL_VAL_ERROR_TO_DATAFRAME,
                                                 orient='index',
                                                 columns=['PHI','PHI_unc','M','M_unc'])

In [29]:
LCBG_FIT_VALS=pd.DataFrame.from_dict(LCBG_VAL_ERROR_TO_DATAFRAME,
                                                 orient='index',
                                                 columns=['PHI','PHI_unc','M','M_unc'])

In [30]:
GAL_FIT_VALS

,PHI,PHI_unc,M,M_unc
10_35,0.006379,0.000163,-20.638894,0.042595
35_55,0.005411,0.000165,-20.769876,0.035128
55_75,0.006415,0.000250,-20.930254,0.034059
75_100,0.006758,0.000305,-21.065415,0.029880
30_80,0.005598,0.000319,-20.900640,0.062364


In [31]:
GAL_FIT_VALS['PHI_ufl']=0
GAL_FIT_VALS['M_ufl']=0

In [32]:
LCBG_FIT_VALS['PHI_ufl']=0
LCBG_FIT_VALS['M_ufl']=0

In [33]:
LCBG_FIT_VALS['PHI_ufl']=LCBG_FIT_VALS.loc[:,'PHI_ufl'].astype(object)
LCBG_FIT_VALS['M_ufl']=LCBG_FIT_VALS.loc[:,'M_ufl'].astype(object)

In [34]:
GAL_FIT_VALS['PHI_ufl']=GAL_FIT_VALS.loc[:,'PHI_ufl'].astype(object)
GAL_FIT_VALS['M_ufl']=GAL_FIT_VALS.loc[:,'M_ufl'].astype(object)

In [35]:
for index in GAL_LFS_INDEX:
    tmpindex=index.split('LF_')[-1]
    print(tmpindex)
    GAL_FIT_VALS.loc[tmpindex,
                     'NGAL']=GAL_LFS[index]['Number_Of_Gals'].sum()
    GAL_FIT_VALS.loc[tmpindex,
                     'NGAL_BELOW_BIAS']=GAL_LFS[index].loc[GAL_LFS[index].Absolute_Magnitude_Bin<GAL_BIAS_CUTOFF[index],'Number_Of_Gals'].sum()
    GAL_FIT_VALS.loc[tmpindex,
                     'WEIGHTED_NGAL']=sum(CATALOG.loc[(CATALOG.Z_BEST>float(tmpindex.split('_')[0])/100)&
                                                      (CATALOG.Z_USE<3)&
                                                      (CATALOG.SG_MASTER==0)&
                                                      (CATALOG.Z_BEST<float(tmpindex.split('_')[1])/100)&
                                                      (CATALOG.subaru_i_mag<22.5)&
                                                      (CATALOG.subaru_i_mag>15.),
                                                      'Spec_Weight'].dropna())
    GAL_FIT_VALS.loc[tmpindex,
                     'PHI_ufl']=uncertainties.ufloat(GAL_FIT_VALS.loc[tmpindex,'PHI'],
                                                        float(GAL_FIT_VALS.loc[tmpindex,'PHI_unc']))
    GAL_FIT_VALS.loc[tmpindex,'M_ufl']=uncertainties.ufloat(float(GAL_FIT_VALS.loc[tmpindex,'M']),float(GAL_FIT_VALS.loc[tmpindex,'M_unc']))
    GAL_FIT_VALS.loc[tmpindex,
                     'INT_NUM_DENS_EIGHTEENFIVE']=integratethings(GAL_FIT_VALS.loc[tmpindex,'PHI_ufl'],
                                                                  GAL_FIT_VALS.loc[tmpindex,'M_ufl'],
                                                                  al=GAL_ALPHA,
                                                                  m=-18.5)
    GAL_FIT_VALS.loc[tmpindex,
                     'INT_NUM_DENS_FIFTEEN']=integratethings(GAL_FIT_VALS.loc[tmpindex,'PHI_ufl'],
                                                                  GAL_FIT_VALS.loc[tmpindex,'M_ufl'],
                                                                  al=GAL_ALPHA,
                                                                  m=-15)
    GAL_FIT_VALS.loc[tmpindex,
                     'ave_z']=(float(tmpindex.split('_')[0])+float(tmpindex.split('_')[1]))/200
    GAL_FIT_VALS.loc[tmpindex,
                     'low_z']=float(tmpindex.split('_')[0])/100
    GAL_FIT_VALS.loc[tmpindex,
                     'high_z']=float(tmpindex.split('_')[1])/100

10_35
35_55
55_75
75_100
30_80


In [40]:
for index in LCBG_LFS_INDEX:
    tmpindex=index.split('LF_')[-1]
    LCBG_FIT_VALS.loc[tmpindex,
                      'NGAL']=LCBG_LFS[index].loc[:,
                                                  'Number_Of_Gals'].sum()
    LCBG_FIT_VALS.loc[tmpindex,
                      'NGAL_BELOW_BIAS']=LCBG_LFS[index].loc[LCBG_LFS[index].Absolute_Magnitude_Bin<LCBG_BIAS_CUTOFF[index],
                                                             'Number_Of_Gals'].sum()
    LCBG_FIT_VALS.loc[tmpindex,
                      'WEIGHTED_NGAL']=sum((CATALOG.loc[(CATALOG.Z_BEST>float(tmpindex.split('_')[0])/100)&
                                                        (CATALOG.Z_USE<3)&
                                                        (CATALOG.SG_MASTER==0)&
                                                        (CATALOG.Z_BEST<float(tmpindex.split('_')[1])/100)&
                                                        (CATALOG.is_LCBG==1)&
                                                        (CATALOG.subaru_i_mag<22.5)&
                                                        (CATALOG.subaru_i_mag>15.),
                                                        'Spec_Weight']*CATALOG.loc[(CATALOG.Z_BEST>float(tmpindex.split('_')[0])/100)&
                                                                                   (CATALOG.Z_USE<3)&
                                                                                   (CATALOG.SG_MASTER==0)&
                                                                                   (CATALOG.Z_BEST<float(tmpindex.split('_')[1])/100)&
                                                                                   (CATALOG.is_LCBG==1)&
                                                                                   (CATALOG.subaru_i_mag<22.5)&
                                                                                   (CATALOG.subaru_i_mag>15.),
                                                                                    'Surface_Brightness_Weight']*CATALOG.loc[(CATALOG.Z_BEST>float(tmpindex.split('_')[0])/100)&
                                                                                                                             (CATALOG.Z_USE<3)&
                                                                                                                             (CATALOG.SG_MASTER==0)&
                                                                                                                             ((CATALOG.Z_BEST<float(tmpindex.split('_')[1]))/100)&
                                                                                                                             (CATALOG.is_LCBG==1)&
                                                                                                                             (CATALOG.subaru_i_mag<22.5)&
                                                                                                                             (CATALOG.subaru_i_mag>15.),
                                                                                                                             'Color_Weight']).dropna())
    LCBG_FIT_VALS.loc[tmpindex,
                      'PHI_ufl']=uncertainties.ufloat(LCBG_FIT_VALS.loc[tmpindex,
                                                              'PHI'],
                                            LCBG_FIT_VALS.loc[tmpindex,
                                                              'PHI_unc'])
    LCBG_FIT_VALS.loc[tmpindex,
                      'M_ufl']=uncertainties.ufloat(LCBG_FIT_VALS.loc[tmpindex,
                                                            'M'],
                                          LCBG_FIT_VALS.loc[tmpindex,
                                                            'M_unc'])
    LCBG_FIT_VALS.loc[tmpindex,
                     'INT_NUM_DENS_EIGHTEENFIVE']=integratethings(LCBG_FIT_VALS.loc[tmpindex,'PHI_ufl'],
                                                                  LCBG_FIT_VALS.loc[tmpindex,'M_ufl'],
                                                                  al=LCBG_ALPHA,
                                                                  m=-18.5)
    LCBG_FIT_VALS.loc[tmpindex,
                     'INT_NUM_DENS_EIGHTEENFIVE_val']=integratethings(LCBG_FIT_VALS.loc[tmpindex,'PHI_ufl'],
                                                                  LCBG_FIT_VALS.loc[tmpindex,'M_ufl'],
                                                                  al=LCBG_ALPHA,
                                                                  m=-18.5).nominal_value
    LCBG_FIT_VALS.loc[tmpindex,
                     'INT_NUM_DENS_EIGHTEENFIVE_err']=integratethings(LCBG_FIT_VALS.loc[tmpindex,'PHI_ufl'],
                                                                  LCBG_FIT_VALS.loc[tmpindex,'M_ufl'],
                                                                  al=LCBG_ALPHA,
                                                                  m=-18.5).std_dev
    LCBG_FIT_VALS.loc[tmpindex,
                     'ave_z']=(float(tmpindex.split('_')[0])+float(tmpindex.split('_')[1]))/200
    LCBG_FIT_VALS.loc[tmpindex,
                     'low_z']=float(tmpindex.split('_')[0])/100
    LCBG_FIT_VALS.loc[tmpindex,
                     'high_z']=float(tmpindex.split('_')[1])/100

In [41]:
LCBG_FIT_VALS

,PHI,PHI_unc,M,M_unc,PHI_ufl,M_ufl,NGAL,NGAL_BELOW_BIAS,WEIGHTED_NGAL,INT_NUM_DENS_EIGHTEENFIVE,INT_NUM_DENS_EIGHTEENFIVE_val,INT_NUM_DENS_EIGHTEENFIVE_err,ave_z,low_z,high_z,Weighted_Fraction_LCBGS,Fraction_LCBGS,WEIGHTED_NGAL_NOMAG
10_35_LCBG,0.001410,0.000068,-20.469905,0.063634,0.00141+/-0.00007,-20.47+/-0.06,319.0,319.0,533.246165,0.00197+/-0.00012,0.001965,0.000118,0.225,0.10,0.35,0.101622,0.079078,533.246165
35_55_LCBG,0.002077,0.000047,-20.554817,0.025851,0.00208+/-0.00005,-20.555+/-0.026,880.0,785.0,1580.104348,0.00303+/-0.00008,0.003034,0.000081,0.450,0.35,0.55,0.285685,0.212406,1580.104348
55_75_LCBG,0.003932,0.000245,-20.494007,0.046844,0.00393+/-0.00025,-20.49+/-0.05,1098.0,935.0,2102.051880,0.0056+/-0.0004,0.005554,0.000375,0.650,0.55,0.75,0.378922,0.275188,2102.051880
75_100_LCBG,0.005133,0.000498,-20.603692,0.054005,0.0051+/-0.0005,-20.60+/-0.05,941.0,894.0,1899.438435,0.0077+/-0.0008,0.007696,0.000779,0.875,0.75,1.00,0.438202,0.314716,1899.438435
30_80,0.002272,0.000202,-20.688162,0.088077,0.00227+/-0.00020,-20.69+/-0.09,2301.0,1507.0,4263.983624,0.00356+/-0.00035,0.003560,0.000355,0.550,0.30,0.80,0.311718,0.228432,4263.983624


In [37]:
LCBG_FIT_VALS['Weighted_Fraction_LCBGS']=LCBG_FIT_VALS.loc[:,'WEIGHTED_NGAL'].values/GAL_FIT_VALS.loc[:,'WEIGHTED_NGAL'].values
LCBG_FIT_VALS['Fraction_LCBGS']=LCBG_FIT_VALS.loc[:,'NGAL'].values/GAL_FIT_VALS.loc[:,'NGAL'].values

In [38]:
LCBG_FIT_VALS.to_csv('./lumfunc_fit_params/lcbg_lumfunc_fit.csv')

In [39]:
GAL_FIT_VALS.to_csv('./lumfunc_fit_params/gal_lumfunc_fit.csv')